In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Integer, Real
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "fChl"]
data.columns

Index(['Turbidity', 'DO', 'fChl', 'Discharge', 'Height', 'WT', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP',
       'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def cvrfmlp(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    mlp = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPRegressor(hidden_layer_sizes=(100,),
                             activation='relu',
                             solver='adam',
                             max_iter=2000,
                             random_state=42))
    ])

    stack = StackingRegressor(
        estimators=[('rf', rf), ('mlp', mlp)],
        final_estimator=RidgeCV(),
        n_jobs=-1
    )

    cv_split = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(stack, X, y, cv=cv_split, n_jobs=-1)

    stack.fit(X, y)

    save_dir = "../models/rfmlp"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{target}.pkl")
    with open(save_path, 'wb') as file:
        pickle.dump(stack, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Hybrid rf–mlp Performance (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
for target in targets:
    cvrfmlp(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/rfmlp\Turbidity.pkl
Hybrid rf–mlp Performance (5-Fold CV):
MAE  = 5.9
RMSE = 8.11
R²   = 94.7 %
MBE  = -0.87
Model saved to ../models/rfmlp\DO.pkl
Hybrid rf–mlp Performance (5-Fold CV):
MAE  = 0.45
RMSE = 0.61
R²   = 90.94 %
MBE  = 0.0
Model saved to ../models/rfmlp\fChl.pkl
Hybrid rf–mlp Performance (5-Fold CV):
MAE  = 1.46
RMSE = 1.94
R²   = 82.56 %
MBE  = -0.23


In [5]:
def cvborfmlp(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    mlp = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPRegressor(max_iter=500, random_state=42))
    ])

    stack = StackingRegressor(
        estimators=[('rf', rf), ('mlp', mlp)],
        final_estimator=RidgeCV(),
        n_jobs=-1
    )

    param_space = {
        'rf__n_estimators': Integer(50, 1000),
        'rf__max_depth': Integer(3, 50),
        'mlp__mlp__hidden_layer_sizes': Integer(50, 200),
        'mlp__mlp__alpha': Real(1e-5, 1e-2, prior='log-uniform'),
        'mlp__mlp__learning_rate_init': Real(1e-4, 1e-2, prior='log-uniform')
    }

    cv_split = KFold(n_splits=5, shuffle=True, random_state=42)

    opt = BayesSearchCV(
        estimator=stack,
        search_spaces=param_space,
        n_iter=100,
        cv=cv_split,
        scoring='r2',
        n_jobs=-1,
        random_state=42,
        verbose=0
    )

    opt.fit(X, y)
    best_model = opt.best_estimator_

    save_dir = "../models/rfmlp"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"bo{target}.pkl")
    with open(save_path, 'wb') as file:
        pickle.dump(best_model, file)
    print(f"Optimized model saved to {save_path}")

    print("Best Parameters:")
    for p, v in opt.best_params_.items():
        print(f"  {p}: {v}")

    y_pred = cross_val_predict(best_model, X, y, cv=cv_split, n_jobs=-1)

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Hybrid rf–mlp Performance (5-Fold CV + BO):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [6]:
for target in targets:
    cvborfmlp(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Optimized model saved to ../models/rfmlp\boTurbidity.pkl
Best Parameters:
  mlp__mlp__alpha: 0.01
  mlp__mlp__hidden_layer_sizes: 117
  mlp__mlp__learning_rate_init: 0.004681087242751113
  rf__max_depth: 50
  rf__n_estimators: 1000
Hybrid rf–mlp Performance (5-Fold CV + BO):
MAE  = 5.34
RMSE = 7.38
R²   = 95.61 %
MBE  = -0.79
Optimized model saved to ../models/rfmlp\boDO.pkl
Best Parameters:
  mlp__mlp__alpha: 0.00370757614120107
  mlp__mlp__hidden_layer_sizes: 142
  mlp__mlp__learning_rate_init: 0.009794324483722095
  rf__max_depth: 49
  rf__n_estimators: 987
Hybrid rf–mlp Performance (5-Fold CV + BO):
MAE  = 0.44
RMSE = 0.6
R²   = 91.41 %
MBE  = -0.01
Optimized model saved to ../models/rfmlp\bofChl.pkl
Best Parameters:
  mlp__mlp__alpha: 0.0007216990952220603
  mlp__mlp__hidden_layer_sizes: 200
  mlp__mlp__learning_rate_init: 0.0019216242367105416
  rf__max_depth: 50
  rf__n_estimators: 1000
Hybrid rf–mlp Performance (5-Fold CV + BO):
MAE  = 1.46
RMSE = 1.93
R²   = 82.58 %
MBE  = -0.